In [ ]:
from objects import Task, Project, DataBase
import datetime


In [4]:
# Step 1: Initialize the database
db = DataBase()

# Step 2: Create a new project
new_project = Project(title="Website Development", description="Tasks for building the new company website")

# Step 3: Add the project to the database
try:
    db.add_project(new_project)
    print(f"Project '{new_project.title}' added to the database.")
except ValueError as e:
    print(e)

# Save the database after adding the project
db.save()

del db

# Step 4: Reload the database later
db = DataBase()
db.load()

# Step 5: Retrieve the project by title
project = db.find_project_by_title("Website Development")

if project:
    print(f"Project '{project.title}' retrieved from the database.")

    # Step 6: Create new tasks
    task1 = Task(title="Design Homepage", notes="Create a homepage wireframe", due_date=datetime(2024, 12, 25))
    task2 = Task(title="Set Up Backend", notes="Build the backend with Django", due_date=datetime(2024, 12, 31))

    # Step 7: Add tasks directly to the project
    project.add_task(task1)
    project.add_task(task2)
    print(f"Added tasks to the project '{project.title}': {task1}, {task2}")

    # Step 8: Save the updated database
    db.save()
else:
    print("Project not found in the database.")


Project 'Website Development' added to the database.
Project 'Website Development' retrieved from the database.


NameError: name 'datetime' is not defined

In [ ]:
import os
import json
import uuid
from datetime import datetime, timedelta
from enum import Enum

def time_to_str(dt: datetime):
    return dt.strftime(r"%Y-%m-%d %H:%M:%S")

def str_to_time(s: str):
    return datetime.strptime(s, r"%Y-%m-%d %H:%M:%S")


class Status(Enum):
    ACTIVE = "ACTIVE"
    DEFERRED = "DEFERRED"
    COMPLETED = "COMPLETED"

class Task:
    def __init__(self, title, notes=None, due_date=None, urgent=False, repeat_behavior=None, dependencies=None, 
                 completed=False, status=Status.ACTIVE, created_at=None, updated_at=None, id=None):
        self.id = id if id else str(uuid.uuid4())
        self.title = title
        self.notes = notes
        self.due_date = due_date
        self.completed = completed
        self.status = status
        self.urgent = urgent
        self.repeat_behavior = repeat_behavior  # e.g., {"interval": timedelta(days=7), "end_date": None}
        self.dependencies = dependencies if dependencies else []
        self.created_at = created_at if created_at else datetime.now()
        self.updated_at = updated_at if updated_at else datetime.now()

    def complete(self):
        if self.dependencies and not all(dep.completed for dep in self.dependencies):
            raise ValueError("Cannot complete task with incomplete dependencies.")
        self.completed = True
        self.status = Status.COMPLETED
        self.updated_at = datetime.now()

        # Handle repeat behavior
        if self.repeat_behavior:
            next_due_date = self.due_date + self.repeat_behavior["interval"] if self.due_date else None
            if not self.repeat_behavior.get("end_date") or next_due_date <= self.repeat_behavior["end_date"]:
                return Task(
                    title=self.title,
                    notes=self.notes,
                    due_date=next_due_date,
                    urgent=self.urgent,
                    repeat_behavior=self.repeat_behavior,
                    dependencies=[]  # Repeated tasks don’t inherit dependencies
                )
        return None

    def defer(self):
        if any(not dep.completed for dep in self.dependencies):
            self.status = Status.DEFERRED
        else:
            self.status = Status.ACTIVE
        self.updated_at = datetime.now()


    def to_dict(self):
        return {
            "id": self.id,
            "title": self.title,
            "notes": self.notes,
            "due_date": time_to_str(self.due_date) if self.due_date else None,
            "completed": self.completed,
            "status": self.status.value,
            "urgent": self.urgent,
            "repeat_behavior": self.repeat_behavior,
            "dependencies": [dep.id for dep in self.dependencies],
            "created_at": time_to_str(self.created_at),
            "updated_at": time_to_str(self.updated_at)
        }

    def __repr__(self):
        return (
            f"Task(id={self.id}, title='{self.title}', completed={self.completed}, status={self.status}, "
            f"due_date={self.due_date}, urgent={self.urgent})"
        )

class Project:
    def __init__(self, title, description=None, tasks=[],
                 id=None, completed=False, created_at=None, updated_at=None):
        self.id = id if id else uuid.uuid4()
        self.title = title
        self.description = description
        self.tasks = tasks
        self.completed = completed if completed else False
        self.created_at = created_at if created_at else datetime.now()
        self.updated_at = updated_at if updated_at else datetime.now()

    def add_task(self, task):
        self.tasks.append(task)
        self.updated_at = datetime.now()

    def complete(self):
        if all(task.completed for task in self.tasks):
            print(f"Project '{self.title}' is complete!")
        else:
            raise ValueError("Cannot complete project with incomplete tasks.")
        
    def to_dict(self):
        return {
            "id": self.id,
            "title": self.title,
            "description": self.description,
            "tasks": [task.id for task in self.tasks],
            "completed": self.completed,
            "created_at": time_to_str(self.created_at),
            "updated_at": time_to_str(self.updated_at)
        }

    def __repr__(self):
        return f"Project(id={self.id}, title='{self.title}', tasks={len(self.tasks)})"


class DataBase:
    def __init__(self):
        self.tasks = []
        self.projects = []

    def add_task(self, task):
        self.tasks.append(task)

    def add_project(self, project):
        self.projects.append(project)

    def save(self, db_path='my_database'):
        with open(os.path.join(db_path, 'tasks.json'), 'w') as f:
            json.dump([task.to_dict() for task in self.tasks], f)

        with open(os.path.join(db_path, 'projects.json'), 'w') as f:
            json.dump([project.to_dict() for project in self.projects], f)

    def load(self, db_path='my_database'):
        with open(os.path.join(db_path, 'tasks.json'), 'r') as f:
            tasks = json.load(f)
            for task in tasks:
                self.add_task(Task(**task))

        with open(os.path.join(db_path, 'projects.json'), 'r') as f:
            projects = json.load(f)
            for project in projects:
                self.add_project(Project(**project))

    def __repr__(self):
        return f"TaskManager(tasks={len(self.tasks)})"

In [8]:
task1 = Task(
    title="Write report",
    notes="Complete the quarterly financial report",
    due_date=datetime(2024, 12, 20),
    urgent=True
)

task2 = Task(
    title="Submit report",
    notes="Submit the report after writing it",
    due_date=datetime(2024, 12, 21),
    dependencies=[task1]
)

my_tm = DataBase()
my_tm.add_task(task1)
my_tm.add_task(task2)
my_tm.save()

In [9]:
new_tm = DataBase()
new_tm.load()

In [56]:
new_tm.tasks[0].__dict__

{'id': '856a56da-57f4-4a0f-b858-142cc568cf0e',
 'title': 'Write report',
 'notes': 'Complete the quarterly financial report',
 'due_date': '2024-12-20 00:00:00',
 'completed': False,
 'status': 'ACTIVE',
 'urgent': True,
 'repeat_behavior': None,
 'dependencies': [],
 'created_at': '2024-12-13 16:39:40',
 'updated_at': '2024-12-13 16:39:40'}

In [57]:
[type(x) for x in task2.to_dict().values()]

[str, str, str, str, bool, str, bool, NoneType, list, str, str]

In [22]:
# # Example Usage
# def main():
#     # Create a task
#     task1 = Task(
#         title="Write report",
#         notes="Complete the quarterly financial report",
#         due_date=datetime(2024, 12, 20),
#         urgent=True
#     )

#     # Create a dependent task
#     task2 = Task(
#         title="Submit report",
#         notes="Submit the report after writing it",
#         due_date=datetime(2024, 12, 21),
#         dependencies=[task1]
#     )

#     # Defer task2 because it depends on task1
#     task2.defer()

#     # Complete task1
#     task1.complete()

#     # Update task2 status
#     task2.defer()

#     # Complete task2
#     task2.complete()

#     # Create a project
#     project = Project(title="Quarterly Report Project", description="Prepare and submit the quarterly report")
#     project.add_task(task1)
#     project.add_task(task2)

#     # Attempt to complete the project
#     project.complete()

#     print(project)

# if __name__ == "__main__":
#     main()
